In [1]:
with open('pagecontent.txt', 'r') as file:
    # Read the content of the file
    content = file.read()

filtered_content = "\n".join(line for line in content.splitlines() if line.strip())

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
import openai

openai.api_key = OPENAI_API_KEY
def get_completion(prompt, model="gpt-4"):
    messages = [{"role":"user","content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message['content']


In [4]:
context = f"""{filtered_content}"""

prompt = f"""
Article: {context}

You will generate increasingly concise, entity-dense summaries of the above Article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative Entities ("; " delimited) from the Article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:
- Relevant: to the main story.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the Article.
- Anywhere: located anywhere in the Article.

Guidelines:
- The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.

Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
"""

response = get_completion(prompt)


'[\n  {\n    "Missing_Entities": "Large Language Model (LLM) agents; financial analysts; retrieval augmented generation (RAG) pipeline",\n    "Denser_Summary": "This extensive and thought-provoking narrative speaks on the topic of Large Language Model (LLM) agents, which are utilized by financial analysts to dissect data, especially about financial performance of companies. Particularly, it emphasizes the combined usage of these LLM agents with retrieval augmented generation (RAG) pipelines to enable analysts to rectify complex questions which could otherwise consume a significant amount of time in data assembly and interpretation."\n  },\n  {\n    "Missing_Entities": "Agent core; Memory module; Tools; Planning module", \n    "Denser_Summary": "The article unpacks the complex architecture of Large Language Model (LLM) agents, in particular, the agent core which holds the central logic and behavioral elements; the memory modules which store the agents\' thought process and interactions;

In [10]:
import json

json_data = json.loads(response)

with open("output.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=2) 